In [8]:
""" Importação de bibliotecas e constantes """

# Importar bibliotecas
# from openpyxl import Workbook
import pandas as pd
from openpyxl import load_workbook
import json


In [9]:
""" Constantes """

baseFileName = "CS Grids - 2019.1-2024.1"
basePath = 'B:/timetabling-UENF/Files/data/Processed Data/'
FILE_PATH = basePath + 'Tabelas/' + baseFileName + '.xlsx'
times = [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
days = ["SEG", "TER", "QUA", "QUI", "SEX"]
properties = ["expectedSemester", "subject", "professor", "expectedDemand", "room", "classDescription"]
OUTPUT_PATH = basePath + 'JSON/' + baseFileName + '.json'
OUTPUT_CSV_PATH = basePath + 'CSV/' + baseFileName + '.csv'
OUTPUT_SQL_PATH = basePath + 'SQL/' + baseFileName + '.sql'

In [10]:
""" Semi manual sheet processing
- set headers and hours
classTimes = [
  {id: 1, horaInicio: 8, duracao: 2, sala: "", dia: "SEG", info: ""},
]
"""

def processInfo(info):
  """ 
  expected input: info = "3|Est. Dados I|Tang|25|Bcct||"
  expected output:
  message = {
    expectedSemester: 3,
    subject: "Est. Dados I",
    professor: "Tang",
    expectedDemand: 25,
    room: "Bcct"
    extraInfo: ["", ""]
  }
  expected input: info = "10|Calc. 2||12"
  expected output:
  message = {
    expectedSemester: 3,
    subject: "Est. Dados I",
    professor: null,
    expectedDemand: 12,
    room: nulll
    extraInfo: []
  }
  """
  message = {}
  
  parts = info.split("|")
  propSize = len(properties)
  partsSize = len(parts)
  
  for i, part in enumerate(parts):
    value = None if part == "" else part
    message[properties[i]] = value
  
  for i in range(partsSize, propSize):
    message[properties[i]] = None
  
  return message

def processCell(cell, yearSemester):
  cellInfo = []
  if cell.value is not None:
    infoList = cell.value.strip().split("\n")
    infoList = [subject for subject in infoList if subject != ""]
    for info in infoList:
      cellClassTimes = {}
      cellClassTimes["duration"] = 2
      cellClassTimes["startHour"] = times[cell.row - 1]
      cellClassTimes["day"] = days[cell.column - 2]
      splittedInfo = processInfo(info)
      cellClassTimes.update(yearSemester)
      cellClassTimes.update(splittedInfo)
      cellInfo.append(cellClassTimes)
  return cellInfo

def getYearSemester(title):
  times = title.split("-")[0].split(".")
  yearSemester = {
    "year": int(str(20) + times[0]),
    "semester": times[1]
  }
  return yearSemester

def sheetProcess(worksheet):
  workSheetInfo = []
  yearSemester = getYearSemester(worksheet.title)
  for row in worksheet.iter_rows():
    for cell in row:
      if cell.row > 1 and cell.column > 1:
        cellInfo = processCell(cell, yearSemester)
        if cellInfo:
          workSheetInfo.extend(cellInfo)
  # Print the worksheet name
  # for item in workSheetInfo:
  #   print(item)
  return workSheetInfo

In [11]:
""" pre processar arquivo
- Ler arquivo XLSX de um caminho armazenado em uma constante
- Iterar sobre todas as planilhas do arquivo
"""

# Carregar o arquivo Excel
workbook = load_workbook(FILE_PATH)

# Processar o arquivo

processOnce = True

sheetsJSON = []
for sheet in workbook.sheetnames:
    worksheet = workbook[sheet]
    if processOnce and worksheet.title != "limpo":
        # processOnce = False
        sheetJSON = sheetProcess(worksheet)
        sheetsJSON.extend(sheetJSON)

In [12]:
""" Export `sheetsJSON` data to a JSON file """

# Serializar para uma string JSON
sheetsJSON_str = json.dumps(sheetsJSON, ensure_ascii=False)

# Escrever a string JSON em um arquivo
with open(OUTPUT_PATH, 'w', encoding='utf-8') as file:
    file.write(sheetsJSON_str)

In [13]:
""" Convert `sheetsJSON` variable  into CSV """

def getJSONtoCSV(sheetsJSON):
  # Convertendo o dicionário para um DataFrame
  sheetsCSVdf = pd.DataFrame(sheetsJSON)

  # Convertendo o DataFrame para CSV
  sheetsCSVdf.to_csv(OUTPUT_CSV_PATH, index=False)
  return sheetsCSVdf

sheetsCSV = getJSONtoCSV(sheetsJSON)

In [14]:
""" Exports `sheetsCSVdf` to a SQL file """

def getCSVToSQL(df, tableName = "classTimes"):
  """
  input: receives a sheetsCSVdf DataFrame containing the data:
  duration, startHour, day, year, semester, expectedSemester, subject, professor, expectedDemand, room, classDescription;

  output: a sql file with the data to be inserted into the database

  It should use the header of the DataFrame to create the insert statement,
  then iterate over the rows to create the insert statements
  """
  # with open(OUTPUT_SQL_PATH, "w") as sql_file:

  sql = f"INSERT INTO {tableName} ("
    # Adicionando os nomes das colunas
  sql += ", ".join(df.columns)
  sql += ") VALUES\n"
  for index, row in df.iterrows():
    # Adicionando os valores das colunas
    sql += "(" + ", ".join([f"'{str(i)}'" for i in row]) + "),\n"
  sql = sql[:-2] + ";\n"
  print(sql)
    # sql_file.write(sql)

# Supondo que sheetsCSVdf é o DataFrame que você quer converter
sheetsSQL = getCSVToSQL(sheetsCSV)

INSERT INTO classTimes (duration, startHour, day, year, semester, expectedSemester, subject, professor, expectedDemand, room, classDescription) VALUES
('2', '8', 'TER', '2024', '1', '1', 'Calculo I', 'Rigoberto', '35', 'None', 'None'),
('2', '8', 'TER', '2024', '1', '3', 'Calculo III', 'Oscar', '15', 'None', 'None'),
('2', '8', 'TER', '2024', '1', '5', 'P Estocáticos', 'Guill', '10', 'Inf2', 'None'),
('2', '8', 'QUA', '2024', '1', '1', 'Introd à CC', 'Rivera', '27', 'sala', 'None'),
('2', '8', 'QUA', '2024', '1', '3', 'Arquitet C', 'Bolsista', '30', 'None', 'None'),
('2', '8', 'QUI', '2024', '1', '1', 'Calculo I', 'None', 'None', 'None', 'None'),
('2', '8', 'QUI', '2024', '1', '3', 'Calculo III', 'Oscar', '15', 'None', 'None'),
('2', '8', 'QUI', '2024', '1', '5', 'P Estocáticos', 'Guill', '10', 'Inf2', 'None'),
('2', '8', 'SEX', '2024', '1', '1', 'Calculo I', 'None', 'None', 'None', 'None'),
('2', '8', 'SEX', '2024', '1', '3', 'Arquitet C', 'Bolsista', '30', 'None', 'None'),
('2', '10'